In [1]:
import Pkg
Pkg.activate(".")

"/home/mbesancon/Documents/CTests/column_generation_jump/Project.toml"

In [7]:
using JuMP
import Cbc
import Clp
import JSON
using SparseArrays: spzeros

In [4]:
const res = open("data0.json", "r") do f
    data = read(f, String)
    JSON.Parser.parse(data)
end

const maxwidth = res["maxwidth"]
const cost = res["cost"]
const prices = Float64.(res["prices"])
const widths = Float64.(res["widths"])
const demand = Float64.(res["demand"])
const nwidths = length(prices);

In [33]:
"""
    subproblem tries to find the best feasible pattern 
    maximizing reduced cost and respecting max roll width
    corresponding to a multiple-item knapsack
"""
function subproblem(reduced_costs, sizes, maxcapacity)
    subm = Model(with_optimizer(Cbc.Optimizer, LogLevel = 0))
    n = length(reduced_costs)
    xs = @variable(subm, xs[1:n] >= 0, Int)
    @constraint(subm, sum(xs.*sizes) <= maxcapacity)
    @objective(subm, Max, sum(xs.*reduced_costs))
    optimize!(subm)
    return round.(Int,JuMP.value.(xs)), round(Int, JuMP.objective_value(subm))
end

subproblem

In [34]:
function init_master(maxwidth, widths, rollcost, demand, prices)
    n = length(widths)
    ncols = length(widths)
    patterns = spzeros(UInt16,n,ncols)
    for i in 1:n
        patterns[i,i] = min(floor(Int,maxwidth/widths[i]),round(Int,demand[i]))
    end
    m = Model(with_optimizer(Clp.Optimizer, LogLevel = 0))
    θ = @variable(m, θ[1:ncols] >= 0)
    @objective(m, Min,
        sum(θ[p]*(rollcost - sum(patterns[j,p]*prices[j] for j=1:n)) for p in 1:ncols)
    )
    @constraint(m, demand_satisfaction[j=1:n], sum(patterns[j,p]*θ[p] for p in 1:ncols) >= demand[j])
    optimize!(m)
    if termination_status(m) != MOI.OPTIMAL
        @warn("No optimal solution")
    end
    return (m, JuMP.value.(θ), demand_satisfaction, patterns)
end

init_master (generic function with 1 method)

In [35]:
(m, θ, demand_satisfaction, patterns) = init_master(maxwidth, widths, cost, demand, prices);

In [36]:
reduced_costs = JuMP.dual.(demand_satisfaction) + prices;

In [37]:
newcol, newobj = subproblem(reduced_costs, widths, maxwidth)

Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -LogLevel 0 -solve -quit (default strategy 1)


([2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0], 1193)

In [38]:
netcost = (cost-sum(newcol[j]*(JuMP.dual(demand_satisfaction[j])+prices[j]) for j in 1:nwidths))

-443.18181818181824

In [39]:
function column_generation(maxwidth, widths, rollcost, demand, prices; maxcols = 5000)
    (m, θ, demand_satisfaction, patterns) = init_master(maxwidth, widths, rollcost, demand, prices)
    ncols = nwidths
    while ncols <= maxcols
        reduced_costs = JuMP.dual.(demand_satisfaction) + prices
        newcol, newobj = subproblem(reduced_costs, widths, maxwidth)
        netcost = cost - sum(newcol[j]*(JuMP.dual.(demand_satisfaction)[j]+prices[j]) for j in 1:nwidths)
        @info "New reduced cost: $netcost"
        if netcost >= 0.0
            return (MOI.OPTIMAL, patterns, JuMP.value.(θ))
        end
        patterns = hcat(patterns, newcol)
        ncols += 1
        m = Model(with_optimizer(Clp.Optimizer, LogLevel = 0))
        θ = @variable(m, θ[1:ncols] >= 0)
        @objective(m, Min,
            sum(θ[p]*(rollcost - sum(patterns[j,p]*prices[j] for j=1:nwidths)) for p in 1:ncols)
        )
        @constraint(m, demand_satisfaction[j=1:nwidths], sum(patterns[j,p]*θ[p] for p in 1:ncols)>=demand[j])
        optimize!(m)
        if termination_status(m) != MOI.OPTIMAL
            @warn("No optimal")
            return (termination_status(m), patterns, getvalue(θ))
        end
    end
    return (:NotFound, patterns, :NoVariable)
end

column_generation (generic function with 1 method)

In [40]:
status, patterns, θ = column_generation(maxwidth, widths, cost, demand, prices, maxcols = 500);
status

Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -LogLevel 0 -solve -quit (default strategy 1)
Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -LogLevel 0 -solve -quit (default strategy 1)
Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -LogLevel 0 -solve -quit (default strategy 1)
Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -LogLevel 0 -solve -quit (default strategy 1)
Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -LogLevel 0 -solve -quit (default strategy 1)
Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -LogLevel 0 -solve -quit (default strategy 1)
Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C

┌ Info: New reduced cost: -443.18181818181824
└ @ Main In[39]:8
┌ Info: New reduced cost: -375.0
└ @ Main In[39]:8
┌ Info: New reduced cost: -264.0
└ @ Main In[39]:8
┌ Info: New reduced cost: -250.0
└ @ Main In[39]:8
┌ Info: New reduced cost: -187.5
└ @ Main In[39]:8
┌ Info: New reduced cost: -150.0
└ @ Main In[39]:8
┌ Info: New reduced cost: -150.0
└ @ Main In[39]:8
┌ Info: New reduced cost: -107.14285714285711
└ @ Main In[39]:8


Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -LogLevel 0 -solve -quit (default strategy 1)
Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -LogLevel 0 -solve -quit (default strategy 1)
Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -LogLevel 0 -solve -quit (default strategy 1)
Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -LogLevel 0 -solve -quit (default strategy 1)


┌ Info: New reduced cost: -97.5
└ @ Main In[39]:8
┌ Info: New reduced cost: -107.14285714285734
└ @ Main In[39]:8
┌ Info: New reduced cost: -72.0
└ @ Main In[39]:8
┌ Info: New reduced cost: -53.571428571428555
└ @ Main In[39]:8


Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -LogLevel 0 -solve -quit (default strategy 1)
Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -LogLevel 0 -solve -quit (default strategy 1)
Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -LogLevel 0 -solve -quit (default strategy 1)
Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -LogLevel 0 -solve -quit (default strategy 1)


...

Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -LogLevel 0 -solve -quit (default strategy 1)
Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -LogLevel 0 -solve -quit (default strategy 1)


┌ Info: New reduced cost: -5.846153846153811
└ @ Main In[39]:8
┌ Info: New reduced cost: -6.0606060606060055
└ @ Main In[39]:8


Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -LogLevel 0 -solve -quit (default strategy 1)
Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -LogLevel 0 -solve -quit (default strategy 1)


┌ Info: New reduced cost: -0.5069124423963558
└ @ Main In[39]:8
┌ Info: New reduced cost: 0.0
└ @ Main In[39]:8


OPTIMAL::TerminationStatusCode = 1

In [41]:
# take worse case from linear solution, round up 
intial_integer = ceil.(Int,θ);
println(sum(θ))

446.1


In [42]:
"""
    From patterns built in the column generation phase, find an integer solution
"""
function branched_model(patterns, demand, rollcost, prices; npatts = size(patterns)[2], initial_point = zeros(Int,npatts))
    npatts = size(patterns)[2]
    m = Model(with_optimizer(Cbc.Optimizer, LogLevel = 0))
    θ = @variable(m, θ[p = 1:npatts] >= 0, Int, start = initial_point[p])
    @objective(m, Min,
        sum(θ[p]*(rollcost - sum(patterns[j,p]*prices[j] for j=1:nwidths)) for p in 1:npatts)
    )
    @constraint(m, demand_satisfaction[j=1:nwidths], sum(θ[p]*patterns[j,p] for p in 1:npatts) >= demand[j])
    optimize!(m)
    return (termination_status(m), round.(Int,(JuMP.value.(θ))))
end

branched_model

In [43]:
status, θ_final = branched_model(patterns, demand, cost, prices; initial_point = intial_integer)

┌ Warning: MathOptInterface.VariablePrimalStart() is not supported by MathOptInterface.Bridges.LazyBridgeOptimizer{MathOptInterface.Utilities.CachingOptimizer{Cbc.Optimizer,MathOptInterface.Utilities.UniversalFallback{JuMP._MOIModel{Float64}}},MathOptInterface.Utilities.UniversalFallback{MathOptInterface.Bridges.AllBridgedConstraints{Float64}}}. This 
│   information will be discarded. = information will be discarded.
└ @ MathOptInterface.Utilities /home/mbesancon/.julia/packages/MathOptInterface/C3lip/src/Utilities/copy.jl:133


Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -LogLevel 0 -solve -quit (default strategy 1)


(OPTIMAL::TerminationStatusCode = 1, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 2, 44, 43, 55, 19, 0, 5, 5, 1])

In [44]:
Int.(patterns)

16×45 SparseArrays.SparseMatrixCSC{Int64,Int64} with 88 stored entries:
  [1 ,  1]  =  22
  [2 ,  2]  =  7
  [3 ,  3]  =  5
  [4 ,  4]  =  4
  [5 ,  5]  =  4
  [6 ,  6]  =  3
  [7 ,  7]  =  3
  [8 ,  8]  =  3
  [9 ,  9]  =  2
  [10, 10]  =  2
  [11, 11]  =  2
  [12, 12]  =  2
  ⋮
  [12, 41]  =  1
  [2 , 42]  =  5
  [8 , 42]  =  1
  [2 , 43]  =  1
  [8 , 43]  =  1
  [14, 43]  =  1
  [6 , 44]  =  1
  [8 , 44]  =  1
  [11, 44]  =  1
  [6 , 45]  =  1
  [9 , 45]  =  1
  [10, 45]  =  1

In [45]:
# final excess rolls
patterns * θ_final - demand

16-element Array{Float64,1}:
 22.0
  1.0
  0.0
  0.0
  0.0
  2.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [46]:
sum(θ_final)

447